<a href="https://colab.research.google.com/github/sharon504/ml-learning/blob/main/attention_is_all_you_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.nn import functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!kaggle datasets download -d adarshpathak/shakespeare-text
!rm text.txt
!unzip shakespeare-text.zip
!rm shakespeare-text.zip

Dataset URL: https://www.kaggle.com/datasets/adarshpathak/shakespeare-text
License(s): unknown
100% 432k/432k [00:00<00:00, 610kB/s]
100% 432k/432k [00:00<00:00, 609kB/s]
rm: cannot remove 'text.txt': No such file or directory
Archive:  shakespeare-text.zip
  inflating: text.txt                


In [ ]:
with open('text.txt', 'r', encoding='utf-8') as f:
  text = f.read()
stoi = {s: i for i, s in enumerate(sorted(list(set(text))))}
itos = {i: s for s, i in stoi.items()}
encode = lambda s: [stoi[char] for char in s]
decode = lambda a: "".join([itos[i] for i in a])

In [ ]:
batch_size = 256 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 2500
eval_interval = 1
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 250
n_embd = 1024
n_head = 16
n_layer = 16
dropout = 0.2
vocab_size = len(stoi)

In [ ]:
data = torch.tensor(encode(text))

In [ ]:
n = int(0.9 * len(text))
data_train = data[:n]
data_val = data[n:]

In [ ]:
def gen_batch(split):
  data = data_train if split == 'train' else data_val
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = gen_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class SelfAttention(nn.Module):
  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape
    wei = self.query(x) @ self.key(x).transpose(-2, -1) * C ** -0.5 # (B, T, C) @ (B, C, T) --> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    out =  wei @ self.value(x) # (B, T, T) @ (B, T, C) --> (B, T, C)
    return out

In [ ]:
class MaskedMultiAttention(nn.Module):
  def __init__(self, head_size, n_head):
    super().__init__()
    self.head_list = nn.ModuleList([SelfAttention(head_size) for _ in range(n_head)])
    self.proj = nn.Linear(n_embd, n_embd, bias=False)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.head_list], dim=-1)
    out = self.dropout(self.proj(out))
    return out

In [ ]:
class FeedForward(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd, bias=False),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd, bias=False),
        nn.Dropout(dropout),
    )
  def forward(self, x):
    out = self.net(x)
    return out

In [ ]:
class Block(nn.Module):
  def __init__(self):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MaskedMultiAttention(head_size, n_head)
    self.ff = FeedForward()
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ff(self.ln2(x))
    out = x
    return out


In [ ]:
class TransformerModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.input_encoding = nn.Embedding(vocab_size, n_embd, device=device)
    self.pos_encoding = nn.Embedding(block_size, n_embd, device=device)
    self.block = nn.Sequential(*[Block() for _ in range(n_layer)])
    self.l_nm = nn.LayerNorm(n_embd)
    self.h_layer = nn.Linear(n_embd, vocab_size, bias=False, device=device)

  def forward(self, idx, target=None):
    B, T = idx.shape
    h = self.input_encoding(idx)
    x = h + self.pos_encoding(torch.arange(T, device=device))
    x = self.block(x)
    x = self.l_nm(x)
    logits = self.h_layer(x)
    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      target = target.view(B*T)
      logits = logits.view(B*T, C)
      loss = F.cross_entropy(logits, target)
    return logits, loss

  def generate(self, idx, max_new_tokens):
          # idx is (B, T) array of indices in the current context
          for _ in range(max_new_tokens):
              # crop idx to the last block_size tokens
              # print(idx.shape)
              idx_cond = idx[:, -( 2 * block_size):].view(-1, block_size) if idx.shape[0] != 1 else idx[:, -block_size:]
              # get the predictions
              logits, loss = self(idx_cond)
              # focus only on the last time step
              logits = logits[:, -1, :] # becomes (B, C)
              # apply softmax to get probabilities
              probs = F.softmax(logits, dim=-1) # (B, C)
              # sample from the distribution
              idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
              # append sampled index to the running sequence
              idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
          return idx

model = TransformerModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = gen_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

201.674752 M parameters


In [ ]:
import os
torch.save(model, './model.py')

In [ ]:
torch.save(model.state_dict(), './model_state_dict.py')

In [ ]:
ix = torch.randint(len(data_train) - block_size, (batch_size,))
x = torch.stack([data_train[i: i+block_size] for i in ix])

In [ ]:
x[1, -block_size:], x[1, :]

(tensor([ 1, 21,  1, 40, 59, 58,  1, 52, 53, 61,  1, 58, 46, 43,  1, 50, 53, 56,
         42,  1, 53, 44,  1, 57, 59, 41, 46,  1, 46, 53, 58,  1]),
 tensor([ 1, 21,  1, 40, 59, 58,  1, 52, 53, 61,  1, 58, 46, 43,  1, 50, 53, 56,
         42,  1, 53, 44,  1, 57, 59, 41, 46,  1, 46, 53, 58,  1]))